# Basic code

## For problems

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state

    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

## For resolve

### Queues

In [2]:
FIFOQueue = deque

LIFOQueue = list


class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = []  # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self):
        return self.items[0][1]

    def __len__(self):
        return len(self.items)

### Search Algorithms

In [3]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n):
    return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))


def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"

    def find_cycle(ancestor, k):
        return (
            ancestor is not None
            and k > 0
            and (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1))
        )

    return find_cycle(node.parent, k)


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result


def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

## Generate report

In [4]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

# Problems

## Missionaries and cannibals

Tres misioneros y tres canibales se encuentran a un lado de un rio, junto con un bote que soporta una o dos
personas.
Encuentre una forma de llevar a los seis al otro lado del rio de manera que el numero de misioneros en cualquier
orilla del rio nunca sea menor que el numero de canibales con que se encuentren.

In [5]:
class Boat(Problem):
    """A problem to find a best way of move people in `Boat`.
    Create a problem with Boat( (# missionaries, # cannibals) )
    """

    def __init__(self, initial):
        self.__dict__.update(initial=(initial, (0, 0)), goal=((0, 0), initial))

    def actions(self, state):
        (M, C), (M1, C1) = state

        if M == 3 and C == 3:
            return [(0, 1), (0, 2), (1, 1)]
        elif M == 2 and C == 2:
            return [(2, 0), (1, 1)]
        elif M == 0 and C == 2:
            return [(0, 1), (0, 2)]
        elif M == 0 and C == 1:
            return [(0, 1)]
        elif M == 1 and C == 1:
            return [(1, 0), (1, 1)]
        elif M == 0 and C == 0:
            return []
        elif M == 3 and C == 1:
            return [(2, 0)]
        elif M == 3 and C == 2:
            return [(1, 0), (0, 1)]
        elif M == 3 and C == 1:
            return [(2, 0)]
        else:
            print("Invalid state")
            return []

    def h(self, node):
        (M, C), _ = node.state
        return C

    def action_cost(self, s, a, s1):
        return 1

    def result(self, state, action):
        """The result of moving people in the boat."""
        (M, C), (M1, C1) = state

        return ((M - action[0], C - action[1]), (M1 + action[0], C1 + action[1]))

#### Test

In [6]:
# state = ((2, 2), (1, 1))  # [(2,0), (1,1)]
# state = ((3, 3), (0, 0))  # [(0,1), (0,2), (1,1)]
# state = ((0, 2), (3, 1))  # [(0,1), (0,2)]
# state = ((0, 1), (3, 2))  # [(0, 1)]
# state = ((1, 1), (2, 2))  # [(1, 0), (1, 1)]
# state = ((0, 0), (3, 3))  # []
# state = ((3, 1), (0, 2))  # [(2, 0)]
# state = ((3, 2), (0, 1))  # [(1,0),(0, 1)]
# state = ((3, 1), (0, 2))  # [(2,0)]

# print(p1.result(state, p1.actions(state)[0]))

### Run

In [7]:
p1 = Boat((3, 3))

In [8]:
report([astar_search, greedy_bfs, uniform_cost_search, breadth_first_bfs, depth_first_bfs, iterative_deepening_search], [p1])

astar_search:
       10 nodes |        6 goal |    3 cost |       8 actions | Boat(((3, 3), (0, 0)), ((0, 0), (3, 3)))
       10 nodes |        6 goal |    3 cost |       8 actions | TOTAL

greedy_bfs:
        6 nodes |        4 goal |    3 cost |       6 actions | Boat(((3, 3), (0, 0)), ((0, 0), (3, 3)))
        6 nodes |        4 goal |    3 cost |       6 actions | TOTAL

uniform_cost_search:
       13 nodes |        8 goal |    3 cost |      10 actions | Boat(((3, 3), (0, 0)), ((0, 0), (3, 3)))
       13 nodes |        8 goal |    3 cost |      10 actions | TOTAL

breadth_first_bfs:
       13 nodes |        8 goal |    3 cost |      10 actions | Boat(((3, 3), (0, 0)), ((0, 0), (3, 3)))
       13 nodes |        8 goal |    3 cost |      10 actions | TOTAL

depth_first_bfs:
        9 nodes |        6 goal |    3 cost |       8 actions | Boat(((3, 3), (0, 0)), ((0, 0), (3, 3)))
        9 nodes |        6 goal |    3 cost |       8 actions | TOTAL

iterative_deepening_search:
       18

In [11]:
print(path_states(astar_search(p1)))
print(path_states(greedy_bfs(p1)))
print(path_states(uniform_cost_search(p1)))
print(path_states(breadth_first_bfs(p1)))
print(path_states(depth_first_bfs(p1)))
print(path_states(iterative_deepening_search(p1)))

[((3, 3), (0, 0)), ((3, 1), (0, 2)), ((1, 1), (2, 2)), ((0, 0), (3, 3))]
[((3, 3), (0, 0)), ((3, 1), (0, 2)), ((1, 1), (2, 2)), ((0, 0), (3, 3))]
[((3, 3), (0, 0)), ((3, 1), (0, 2)), ((1, 1), (2, 2)), ((0, 0), (3, 3))]
[((3, 3), (0, 0)), ((3, 1), (0, 2)), ((1, 1), (2, 2)), ((0, 0), (3, 3))]
[((3, 3), (0, 0)), ((3, 1), (0, 2)), ((1, 1), (2, 2)), ((0, 0), (3, 3))]
[((3, 3), (0, 0)), ((2, 2), (1, 1)), ((1, 1), (2, 2)), ((0, 0), (3, 3))]


In [12]:
print(path_actions(greedy_bfs(p1)))
print(path_actions(astar_search(p1)))
print(path_actions(uniform_cost_search(p1)))
print(path_actions(breadth_first_bfs(p1)))
print(path_actions(depth_first_bfs(p1)))
print(path_actions(iterative_deepening_search(p1)))

[(0, 2), (2, 0), (1, 1)]
[(0, 2), (2, 0), (1, 1)]
[(0, 2), (2, 0), (1, 1)]
[(0, 2), (2, 0), (1, 1)]
[(0, 2), (2, 0), (1, 1)]
[(1, 1), (1, 1), (1, 1)]
